### **<span style="color:red">DATES AND TIME ZONES</span>**

* TIME ZONE - LOCAL TIME OF A REGION OR COUNTRY.
* GREENWICH MEAN TIME (GMT) - AVERAGE TIME EACH DAY WHEN THE SUN CROSSES THE PRIME MERIDIAN AT THE ROYAL OBSERVATORY GREENWICH.
* UNIVERSAL COORDINATE TIME (UTC) - STANDARD TIME SYSTEM USED TO SET CLOCKS AND TIME ZONES AROUND THE WORLD.

>NOTE: GMT is a time zone. UTC is a time standard.
___

### **<span style="color:red">SETTING UP TIME ZONES</span>**

<span style="color:red">POSTGRES STORES DATES AND TIMES IN UTC AND CONVERTS THEM BASED ON CLIENT CONFIGURATIONS </span>

```sql
SHOW TIME ZONE;
```

RETURNS: America/Los_Angeles

>NOTE: When we installed postgres it aligned it's configuration to my local configuration to show me dates and times in my local time zone.

\
<span style="color:red">WE CAN EXPLICITLY SET THE TIMEZONE TO UTC (FOR ONE SESSION)</span>

```sql
SET TIME ZONE 'UTC';

SHOW TIME ZONE;
```

RETURNS: UTC

>NOTE: When you set the time zone, it is only set for that session.


\
<span style="color:red">WE CAN EXPLICITLY SET THE TIMEZONE TO UTC AT SYSTEM LEVEL (FOR ALL USERS SESSIONS)</span>

```sql
ALTER USER username SET timezone='UTC';

SHOW TIME ZONE;
```

RETURNS: UTC

>NOTE: For windows and linux the user is postgres, for mac the user is your username that you log in with.

>NOTE: You may need to kill your connections and then reconnect to your database.
___

### **<span style="color:red">HOW DO WE FORMAT DATE AND TIME?</span>**

<span style="color:red">ISO-86O1</span>
* FORMATTING STANDARD USED BY POSTGRESQL
* DEPICTS UTC DATE AND TIME IN A UNIFORM WAY

\
<span style="color:red">ISO-8601 FORMAT:</span>

<span style="color:lightgreen">YYYY-MM-DD</span><span style="color:fuchsia">THH:MM:SS</span><span style="color:aqua"> TZ</span>

<span style="color:lightgreen">2017-08-17</span><span style="color:fuchsia">T12:47:16</span><span style="color:aqua">+02:00</span>

<span style="color:lightgreen">The date is August 17, 2017.</span><span style="color:fuchsia"> The time is 12:47:16 UTC.</span><span style="color:aqua"> The time zone is two hours ahead of UTC.</span>

>NOTE: The offset,<span style="color:aqua">+02:00</span> is optional. If it is not given, it is assumed that it is already in UTC.

>NOTE: UTC is a time standard. ISO-8601 is the standard format. A format is a way of representing a date and time.
___

### **<span style="color:red">TIMESTAMPS</span>**

DATES WITH TIME AND TIMEZONE INFO

\
<span style="color:red">VIEW TIMESTAMP</span>

```sql
SELECT now()
```

Reurns: 2023-11-28 06:42:27.332425+00

\
<span style="color:red">STORE TIMESTAMP</span>

```sql
CREATE TABLE timezones(
    ts TIMESTAMP WITHOUT TIME ZONE,
    tz TIMESTAMP WITH TIME ZONE
)

INSERT INTO timezones VALUES(
    TIMESTAMP WITHOUT TIME ZONE '2000-01-01 10:00:00-05', --ignore timezone
    TIMESTAMP WITH TIME ZONE '2000-01-01 10:00:00-05' -- take timezone into account and offset it  
)

SELECT * FROM timezones;
```

Reurns:
|ts|tz|
|:---:|:---:|
|2000-01-01 10:00:00|2000-01-01 15:00:00+00|

\
<span style="color:red">`TIME STAMP WITH TIME ZONE` VS `TIME STAMP WITHOUT TIME ZONE`</span>

* Depends on how much information you want to store.
* The column `ts` contains timestamp without time zone information. Time is not offset and is assumed to be in UTC.
* The column `tz` contains timestamp with time zone information. Time is offset to account for timezone information.

>Note: Columns will be inserted in sequence, meaning in the order in which they were entered.

\
<span style="color:red">SHOULD YOU USE TIME STAMPS OR DATES?</span>

It depends on what you are storing.

It is possible to switch between both, but ideally you want to store it correctly the first time.
___

### **<span style="color:red">POSTGRES GIVES US OPERATORS TO HELP SIMPLIFY DATES</span>**

<span style="color:red">CURRENT DATE</span>


```sql
SELECT NOW(); -- returns current date and time
```

Returns: `2023-12-02 04:35:11.219932+00`

\
We can cast these results to a date type, so that we only extract the date information.

```sql
SELECT NOW()::date; -- casts timestamp to date
```

Returns: `2023-11-28`

\
A more direct approach would be to use the current_date function.
```sql
SELECT CURRENT_DATE; -- returns formatted subset of ISO-8601 date format 'YYYY-MM-DD'
```

Returns: `2023-11-28`

\
<span style="color:red">TO_CHAR</span>

\
You can specify the format you want using format modifiers (d,m,y)...

```sql
SELECT TO_CHAR(CURRENT_DATE, 'dd/mm/yyyy'); -- used to format the current date
```

Returns: `28/11/2023`

\
We can use template format patterns...

```sql
SELECT TO_CHAR(CURRENT_DATE, 'DDD'); -- converts current date to the following format: day of year (001–366)
```

Returns: `332`

>NOTE: See [docs](https://www.postgresql.org/docs/current/functions-formatting.html) for other format templates.

\
<span style="color:red">DIFFERENCE</span>

What is the difference in days between now and the 1800's?

```sql
SELECT NOW() - '1800/01/01'; -- returns number of days, hours, minutes and seconds that have passed 
```

Returns: `81784 days 03:02:07.821834`

>Note: Subtracting dates returns the difference in days.

\
<span style="color:red">TO DATE</span>

What is the difference in days between now and the 1800's?

```sql
SELECT date '1800/01/01'; -- casts string to a date
```

Returns: `1800-01-01`

>Note: Date converts date to a date format for postgres.

\
<span style="color:red">CALCULATE AGE</span>

```sql
SELECT AGE(date '1800/01/01');
```

Returns: `223 years 11 mons 1 days`
 

\
Age can take multiple parameters...

```sql
SELECT AGE(date '1992/11/13', date '1800/01/01');
```

Returns: `192 years 10 mons 12 days`

>Note: Age returns an interval. We may want to cast this to a specific format using the char interval.

\
<span style="color:red">EXTRACT DATE</span>

```sql
SELECT EXTRACT (DAY FROM date '1992/11/13');
```

Returns: `13`

>Note: Don't forget the parentheses. You can do the same thing with the `MONTH` or `YEAR` keywords which will return `11` and `1992` respectively.

\
<span style="color:red">ROUND A DATE</span>


```sql
SELECT DATE_TRUNC('year', date '1992/11/13');
```

Returns: `1992-01-01 00:00:00+00`

>Note: If we used the `month` keyword it would return `1992-11-01 00:00:00+00`.

\
<span style="color:red">INTERVAL</span>

```sql
SELECT *
FROM orders
WHERE purchaseDate <= now() - interval '30 days'>;
```

Returns: returns date of purchase for the last 30 days 

>Note: If the purchase date is less than or equal to the interval of 30 days from the current date (the current date minus 30 days ago), return those purchase dates.

>Note: You can use multiple identifiers (e.g. `INTERVAL '1 year 2 months 3 days;`, `INTERVAL 2 weeks ago`, `INTERVAL 1 year 3 hours 20 minutes;`)

\
<span style="color:red">EXTRACTING WITH INTERVAL</span>

```sql
SELECT
    EXTRACT (
        year
        FROM
            INTERVAL '5 years 20 months'
    );
```

Returns: 6
___

___
___
#### <span style="color:lightgreen">Exercise Date and Timestamp</span>
___
___

<span style="color:lightgreen">EXERCISE 1:</span>

DB: Employees

Table: employees

Question: Get me all the employees above 60, use the appropriate date functions.

\
<span style="color:lightgreen">ANSWER:</span>

```sql
SELECT first_name, last_name, EXTRACT(YEAR FROM AGE(birth_date)) AS age
From employees
WHERE EXTRACT(YEAR FROM AGE(birth_date)) >= '60';
```
\
<span style="color:lightgreen">SOLUTION:</span>

```sql
SELECT AGE(birth_date), * FROM employees
WHERE (
   EXTRACT (YEAR FROM AGE(birth_date))
) > 60 ;

-- alternative
SELECT count(birth_date) FROM employees
WHERE birth_date < now() - interval '61 years' -- 61 years before the current date
```

\
<span style="color:lightgreen">Interval break down...</span>

\
Let's find the current date...

```sql
SELECT NOW()::date; -- Returns: `2023-12-03`
```

\
Now, let's find the date 61 years prior to the current date...

```sql
SELECT (NOW() - interval '61 years')::date; -- Returns: `1962-12-03`
```

\
Anyone born within this interval is 60 years old or younger...

```sql
SELECT AGE(NOW() - INTERVAL '61 years'); -- Returns: `60 years 11 mons 30 days 16:44:05.716733`
```

\
Older birth_date < 1962-12-03 < More recent birth date

Therefore, anyone with a birth date that is older than (`<`) 1962-12-03 (`birth_date < NOW() - INTERVAL '61 years`) is over 60 years old and anyone born after 1962-12-03 (`birth_date > NOW() - INTERVAL '61 years`) is 60 or younger.
___

<span style="color:lightgreen">EXERCISE 2:</span>

DB: Employees

Table: employees

Question: How many employees where hired in February?

\
<span style="color:lightgreen">ANSWER:</span>

```sql
SELECT count(emp_no) 
FROM employees
where EXTRACT (MONTH FROM hire_date) = 2;
```

Returns: `24448`
___

<span style="color:lightgreen">EXERCISE 3:</span>

DB: Employees

Table: employees

Question: How many employees were born in november?

\
<span style="color:lightgreen">ANSWER:</span>

```sql
SELECT COUNT(emp_no) 
FROM employees
WHERE EXTRACT (MONTH FROM birth_date) = 11;
```

Returns: `24500`
___

<span style="color:lightgreen">EXERCISE 4:</span>

DB: Employees

Table: employees

Question: Who is the oldest employee? (Use the analytical function MAX)

\
<span style="color:lightgreen">ANSWER:</span>

How old is the oldest employee?

```sql
SELECT MAX(AGE(birth_date)) FROM employees;
```

\
Returns: `71 years 10 mons 2 days`

\
Who is the oldest employee?

```sql
SELECT * FROM employees
WHERE AGE(birth_date) = (
   SELECT MAX(AGE(birth_date)) 
   FROM employees
);
```

\
Returns: 

|emp_no|birth_date|first_name|last_name|gender|hire_date|
|:---:|:---:|:---:|:---:|:---:|:---:|
|91374|1952-02-01|Eishiro|Kuzuoka|M|1992-02-12|
|237571|1952-02-01|Ronghao|Schaad|M|1988-07-10|
|65308|1952-02-01|Jouni|Pocchiola|M|1985-03-10|
|87461|1952-02-01|Moni|Decaestecker|M|1986-10-06|
|207658|1952-02-01|Kiyokazu|Whitcomb|M|1988-07-26|
|406121|1952-02-01|Supot|Remmele|M|1989-01-27|

___

<span style="color:lightgreen">EXERCISE 5:</span>

DB: Store

Table: orders

Question: How many orders were made in January 2004?

\
<span style="color:lightgreen">ANSWER:</span>

```sql
SELECT COUNT(orderid)
FROM orders
WHERE EXTRACT(MONTH FROM orderdate) = 1 AND EXTRACT(YEAR FROM orderdate) = 2004;
```

\
Returns: `1000`

\
<span style="color:lightgreen">ANSWER:</span>

```sql
SELECT COUNT(orderid)
FROM orders
WHERE TO_CHAR(orderdate, 'MM/YYYY')::text = '01/2004';
```

\
Returns: `1000`

\
<span style="color:lightgreen">ANSWER:</span>

```sql
SELECT COUNT(orderid)
FROM orders
WHERE DATE_TRUNC('month', orderdate) = date '2004-01-01';
```

\
Returns: `1000`
___